# Movie Review (MR)
Set notebook to read SQL commands.

In [1]:
# Import basic libraries
import pandas as pd
import sqlite3

In [2]:
!pip install prettytable==2.5.0

In [3]:
# Import movie databases
df1 = pd.read_csv('title.basics.tsv.gz', sep='\t')
df2 = pd.read_csv('title.crew.tsv.gz', sep='\t')
df3 = pd.read_csv('title.ratings.tsv.gz', sep='\t')

<ipython-input-3-b72dd7853022>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('title.basics.tsv.gz', sep='\t')


In [4]:
# Connect to sql
conn = sqlite3.connect('movie.db')

# Transform csv in db
df1.to_sql('basic', conn, index = False, if_exists = 'replace')
df2.to_sql('crew', conn, index = False, if_exists = 'replace')
df3.to_sql('rating', conn, index = False, if_exists = 'replace')


1503482

In [5]:
# Load sql
## Run in Google colab
%load_ext sql
## Run in your local idle
#%load_ext sqlite3

# Connect to sql server
%sql sqlite:///movie.db

## Business Questions

### 1. Genre Popularity Analysis
- What are the top 5 most popular genres based on the number of films released?
- How does the average movie rating vary by genre?
- Which genres saw the biggest increase in the number of releases over the last 5 years?

SQL Skills:
- Joins (linking title.basics with genre-related tables).
- Aggregation functions (`COUNT()`, `AVG()`).
- Subqueries (for filtering by recent years).
- Sorting (`ORDER BY`).

In [23]:
# What are the top 5 most popular genres based on the number of films released?
%%sql
SELECT genres, COUNT(genres) as count_genres
FROM basic
GROUP BY genres
ORDER BY count_genres DESC
LIMIT 5;

 * sqlite:///movie.db
Done.


genres,count_genres
Drama,1276745
Comedy,741025
Talk-Show,703731
News,586493
Documentary,540870


In [24]:
# How does the average movie rating vary by genre?
%%sql
SELECT genres, AVG(averageRating) as avg_rating
FROM basic
JOIN rating
On basic.tconst = rating.tconst
GROUP BY genres
ORDER BY avg_rating DESC
LIMIT 10; -- adding a limit so we can see better, but if you remove it you can see the entire list

 * sqlite:///movie.db
Done.


genres,avg_rating
"Documentary,Musical,Reality-TV",9.5
"Animation,Musical,Reality-TV",9.4
"Reality-TV,Short,Talk-Show",9.254761904761901
"Music,War",9.25
"Comedy,Game-Show,Musical",9.21818181818182
"Biography,Crime,Reality-TV",9.216666666666667
"Family,Game-Show,Romance",9.200000000000001
"Mystery,Sci-Fi,Talk-Show",9.2
"History,Music,News",9.2
"Family,History,Talk-Show",9.2


In [25]:
# Now just checking the bottom ratings
%%sql
SELECT genres, AVG(averageRating) as avg_rating
FROM basic
JOIN rating
On basic.tconst = rating.tconst
GROUP BY genres
ORDER BY avg_rating ASC
LIMIT 3;

 * sqlite:///movie.db
Done.


genres,avg_rating
"Drama,Family,Game-Show",1.7
"Biography,Reality-TV,Sport",2.3
"Comedy,Sport,War",2.5


In [26]:
# Which genres saw the biggest increase in the number of releases over the last 5 years?
%%sql
SELECT genres, COUNT(genres) as count_genres
FROM basic
WHERE startYear >= DATE('now')-5
GROUP BY genres
ORDER BY count_genres DESC
LIMIT 5;

 * sqlite:///movie.db
Done.


genres,count_genres
Drama,647329
Talk-Show,293744
Comedy,253035
News,219342
Documentary,177488


### 2. Director Productivity Analysis
- Who are the directors with the most films in the catalog?
- What is the average rating for movies directed by each of the top 10 directors?
- Which director worked across the most genres?

SQL Skills:
- Joins (linking title.crew with title.basics).
- Aggregation functions (`COUNT()`, `AVG()`).
- Subqueries (to rank directors by number of movies).
- Grouping and filtering (`GROUP BY`, `HAVING`).

In [28]:
# Who are the directors with the most films in the catalog?
%%sql
SELECT directors, COUNT(directors) as count_directors
FROM crew
WHERE directors IS NOT '\N'
GROUP BY directors
ORDER BY count_directors DESC
LIMIT 3;

 * sqlite:///movie.db
Done.


directors,count_directors
nm1203430,11449
nm1409127,10363
nm13220986,8024


In [29]:
# What is the average rating for movies directed by each of the top 10 directors?
%%sql
SELECT directors, AVG(averageRating) as avg_rating
FROM crew
JOIN rating ON crew.tconst = rating.tconst
GROUP BY directors
ORDER BY avg_rating DESC
LIMIT 10;

 * sqlite:///movie.db
Done.


directors,avg_rating
nm9948766,10.0
nm9937122,10.0
nm9902552,10.0
"nm9884248,nm2481797,nm4492923",10.0
nm9845861,10.0
nm9833982,10.0
nm9801135,10.0
nm9659299,10.0
"nm9646534,nm11107253",10.0
nm9612258,10.0


In [30]:
# Which director worked across the most genres?
%%sql
SELECT directors, COUNT(genres) as gen_count
FROM crew
JOIN basic ON crew.tconst = basic.tconst
WHERE directors IS NOT '\N' AND genres IS NOT '\N'
GROUP BY directors
ORDER BY gen_count DESC
LIMIT 5;

 * sqlite:///movie.db
Done.


directors,gen_count
nm1203430,11141
nm1409127,10353
nm13220986,8024
"nm5236281,nm5239804",7434
nm0022750,6834


### 3. Yearly Production Trends
- Which year had the most film releases in total?
- What is the trend of movie releases over the last 10 years?
- How many movies and TV series were released in the same year?

SQL Skills:
- Date manipulation (using startYear and endYear).
- Aggregation functions (`COUNT()`).
- Subqueries (for year-over-year comparisons).
- Sorting and filtering (`GROUP BY`).

In [13]:
# Which year had the most film releases in total?
%%sql
SELECT startYear, COUNT(startYear) as releases
FROM basic
WHERE startYear IS NOT '\N'
GROUP BY startYear
ORDER BY releases DESC
LIMIT 5;

 * sqlite:///movie.db
Done.


startYear,releases
2021,499905
2022,479505
2018,453382
2019,448853
2017,447240


In [14]:
# What is the trend of movie releases over the last 10 years?
%%sql
SELECT startYear, COUNT(genres) AS releases, genres
FROM  basic
WHERE startYear IS NOT '\N' AND startYear BETWEEN DATE('now') - 10 AND DATE('now')
GROUP BY startYear
ORDER BY startYear DESC;

 * sqlite:///movie.db
Done.


startYear,releases,genres
2024,326613,Drama
2023,440555,Short
2022,479505,"Comedy,Drama,Music"
2021,499905,Short
2020,430596,"Documentary,Short"
2019,448853,"Action,Crime"
2018,453382,Drama
2017,447240,"Comedy,Drama,Fantasy"
2016,423535,Short
2015,399092,"Comedy,Drama"


In [15]:
# How many movies and TV series were released in the same year?
%%sql
SELECT
  startYear,
  SUM( CASE WHEN titleType LIKE 'movie' THEN 1 END) AS movie_count,
  SUM( CASE WHEN titleType LIKE 'tvseries' THEN 1 END) AS show_count
FROM basic
WHERE startYear IS NOT '\N' AND startYear BETWEEN DATE('now') - 5 AND DATE('now')
GROUP BY startYear
ORDER BY startYear DESC
LIMIT 5;

 * sqlite:///movie.db
Done.


startYear,movie_count,show_count
2024,17512,6605
2023,20106,9011
2022,20558,10384
2021,18886,11911
2020,16491,12476
2019,19210,11352
2018,19260,11527
2017,19025,11549
2016,18562,11294
2015,17372,11072


### 4. Movie Rating Trends
- What are the top 5 highest-rated movies?
- Which genre has the highest average movie rating?
- Is there a correlation between movie runtime and average rating?

SQL Skills:
- Joins (linking title.basics and title.ratings).
- Aggregation functions (`AVG()`, `MAX()`).
- Subqueries (for filtering by ratings).
- Sorting (`ORDER BY`).

In [16]:
# What are the top 5 highest-rated movies?
%%sql
SELECT primaryTitle, averageRating, startYear, genres
FROM basic
JOIN rating ON basic.tconst = rating.tconst
WHERE basic.titleType LIKE 'movie'
ORDER BY averageRating DESC
LIMIT 5;

 * sqlite:///movie.db
Done.


primaryTitle,averageRating,startYear,genres
All I Know Is,10.0,1992,Drama
Em Busca da Ilha Desconhecida,10.0,2001,Documentary
Kaputol,10.0,2019,Drama
Rainy in Glenageary,10.0,2019,Drama
Love Live! Series 9th Anniversary LOVE LIVE! FEST,10.0,2020,Music


In [17]:
# Which genre has the highest average movie rating?
%%sql
SELECT genres, MAX(averageRating) AS max_rating
FROM basic
JOIN rating ON basic.tconst = rating.tconst
WHERE basic.titleType LIKE 'movie' AND genres IS NOT '\N'
GROUP BY genres
ORDER BY max_rating DESC
LIMIT 3;

 * sqlite:///movie.db
Done.


genres,max_rating
Thriller,10.0
Reality-TV,10.0
"Mystery,Thriller",10.0


In [18]:
# Is there a correlation between movie runtime and average rating?
%%sql
SELECT AVG(averageRating) AS avg_rating, runtimeMinutes, genres
FROM basic
JOIN rating ON basic.tconst = rating.tconst
WHERE basic.titleType LIKE 'movie' and runtimeMinutes IS NOT '\N'
GROUP BY runtimeMinutes
ORDER BY avg_rating DESC
LIMIT 5;

 * sqlite:///movie.db
Done.


avg_rating,runtimeMinutes,genres
10.0,8,Music
9.45,2,Music
9.4,379,Documentary
9.1,607,Documentary
9.0,286,Documentary


### 5. TV Series and Movie Comparison
- What is the average rating of TV series compared to movies?
- Which TV series has the longest runtime and how does it compare to movies of similar genres?
- What percentage of releases in the last 5 years were TV series?

SQL Skills:
- Joins (linking title.basics, title.episode).
- Aggregation functions (`AVG()`, `COUNT()`).
- Date manipulation (using startYear and endYear).
- Sorting (`ORDER BY`) and filtering (`HAVING`).

In [19]:
# What is the average rating of TV series compared to movies?
%%sql
SELECT titleType,  ROUND(AVG(averageRating), 2) AS avg_rating
from rating
JOIN basic ON rating.tconst = basic.tconst
WHERE titleType IN ('movie', 'tvSeries')
GROUP BY titleType;

 * sqlite:///movie.db
Done.


titleType,avg_rating
movie,6.17
tvSeries,6.85


In [20]:
# Which TV series has the longest runtime and how does it compare to movies of similar genres?
%%sql
WITH longest_tvshow AS (
  SELECT
    b.genres,
    b.runtimeMinutes AS tv_runtime,
    b.primaryTitle AS series_name
  FROM basic b
  WHERE b.titleType LIKE 'tvseries' AND b.runtimeMinutes IS NOT '\N'
  ORDER BY b.runtimeMinutes DESC
  LIMIT 1
)
SELECT
  b.genres,
  b.primaryTitle AS movie_name,
  b.runtimeMinutes AS movie_runtime,
  l.series_name AS longest_tv_series,
  l.tv_runtime AS longest_tv_runtime
FROM basic b
JOIN longest_tvshow l ON b.genres = l.genres
WHERE b.titleType LIKE 'movie' AND b.runtimeMinutes IS NOT '\N'
GROUP BY b.genres
ORDER BY l.tv_runtime DESC;


 * sqlite:///movie.db
Done.


genres,movie_name,movie_runtime,longest_tv_series,longest_tv_runtime
Drama,The Prodigal Son,90,Shôkin Kasegi,990


In [21]:
# What percentage of releases in the last 5 years were TV series?
%%sql
SELECT
    startYear,
    SUM( CASE WHEN titleType LIKE 'tvseries' THEN 1 END) as tv_count,
    COUNT(titleType) AS total_releases,
    ROUND(100.00 * (SUM( CASE WHEN titleType LIKE 'tvseries' THEN 1 END) *1.0/COUNT(titleType)),2) as tv_percentage
FROM basic
WHERE startYear BETWEEN DATE('now') - 5 AND DATE('now')
GROUP BY startYear
ORDER BY startYear DESC

 * sqlite:///movie.db
Done.


startYear,tv_count,total_releases,tv_percentage
2024,6605,326613,2.02
2023,9011,440555,2.05
2022,10384,479505,2.17
2021,11911,499905,2.38
2020,12476,430596,2.9
2019,11352,448853,2.53
